<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
import numpy as np

np.random.seed(812)

X = np.array(([0, 0, 1], 
              [0, 1, 1],
              [1, 0, 1],
              [0, 1, 0],
              [1, 0, 0],
              [1, 1, 1], 
              [0, 0, 0]), dtype=float)

y = np.array(([0],
              [1],
              [1],
              [1],
              [1],
              [0],
              [0]), dtype=float)

In [2]:
class NeuralNetwork:
    def __init__(self, inputs=3, hiddenNodes=4, outputNodes=1):
        self.inputs = inputs
        self.hiddenNodes = hiddenNodes
        self.outputNodes = outputNodes
        
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes)
        
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def feed_forward(self, x):
        self.hidden_sum = np.dot(X, self.weights1)
        
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
        
    def backward(self, X, y, output):
        self.output_error = y - output
        self.output_delta = self.output_error * self.sigmoidPrime(output)
        
        self.z2_error = self.output_delta.dot(self.weights2.T)
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.activated_hidden)
        
        self.weights1 += X.T.dot(self.z2_delta)
        self.weights2 += self.activated_hidden.T.dot(self.output_delta)
    
    def train(self, X, y):
        output = self.feed_forward(X)
        self.backward(X, y, output)

In [3]:
# Test 
nn = NeuralNetwork()

print(X[0])
output = nn.feed_forward(X[0])
print(output)

[0. 0. 1.]
[[0.79059631]
 [0.8117309 ]
 [0.68827527]
 [0.78234695]
 [0.66062995]
 [0.71709499]
 [0.75622518]]


In [4]:
nn = NeuralNetwork()

for i in range(1000):
    nn.train(X, y)
    
print('Results after Epoch 1000: \n')
print('Loss: \n', str(np.mean(np.square(y - nn.feed_forward(X)))))
print('Predicted Output: \n', str(nn.feed_forward(X)))

Results after Epoch 1000: 

Loss: 
 0.09822540359164399
Predicted Output: 
 [[0.02846269]
 [0.91358458]
 [0.6664282 ]
 [0.98389948]
 [0.67839077]
 [0.67950436]
 [0.05110806]]


## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [66]:
from sklearn.datasets import load_digits
digits = load_digits()

In [67]:
X = digits.data
y = digits.target.reshape(-1,1)

In [68]:
X.shape, y.shape

((1797, 64), (1797, 1))

In [69]:
y = y / 9

In [70]:
X = X / 16

In [71]:
ones = np.ones((X.shape[0], 1))
X_bias = np.hstack((X, ones))
X_bias.shape

(1797, 65)

In [79]:
nn_new = NeuralNetwork(64,65,1)
print(nn_new.inputs)
print(nn_new.hiddenNodes)
print(nn_new.outputNodes)
print(nn_new.weights1.shape)
print(X_bias.shape)

64
65
1
(64, 65)
(1797, 65)


In [81]:
for i in range(10000):
    nn_new.train(X=X, y=y)
    
print('Results after Epoch 10000: \n')
print('Loss: \n', str(np.mean(np.square(y - nn_new.feed_forward(X)))))
print('Predicted Output: \n', str(nn_new.feed_forward(X)))

Results after Epoch 1000: 

Loss: 
 0.3523224578687387
Predicted Output: 
 [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


In [82]:
nn_bias = NeuralNetwork(64,66,1)

for i in range(10000):
    nn_bias.train(X=X_bias, y=y)
    
print('Results after Epoch 10000: \n')
print('Loss: \n', str(np.mean(np.square(y - nn_bias.feed_forward(X_bias)))))
print('Predicted Output: \n', str(nn_bias.feed_forward(X_bias)))

ValueError: operands could not be broadcast together with shapes (64,66) (65,66) (64,66) 